<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:


      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-06-06 09:33:45
-------------------
qualified stocks: 93
with latest results: 31
still star stocks: 17
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  1.64 L
Current:  1.32 C
-------------------
Today PnL: 16.95 K (0.13%)
Current PnL: -16.20 L (-11.29%)
CY Booked + Current PnL: -11.20 L (-7.81%)
-------------------
Total profit:  4.37 L
Total loss:  -20.56 L
-------------------
Total Booked + Current PnL: 15.78 L (13.54%)
Total Booked PnL: 31.97 L (27.43%)
Curr Year Booked PnL: 4.99 L (3.77%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.12 C
Est FTT PnL: 79.09 L (59.73%)
-------------------
Est LTT:  2.74 C
Est LTT PnL: 1.41 C (106.68%)
Deployed:  1.17 C
Current:  1.32 C
CAGR/XIRR %: 9.08%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LICI,785.05,983.0,-2.96,H-LC,89.25,237609.0,42917.0,6178.0,0.16,22.04,2.60,25.21,31.0,6.95,1.82,31.39,XY25,NTT,INSURANCE
4,ADANIPORTS,1103.69,1583.0,3.60,M-LC,63.98,103802.0,25440.0,8595.0,0.36,32.46,8.28,43.43,70.0,2.96,0.79,38.83,XY24,NTT,MISC
62,RELIANCE,1291.83,1526.0,5.74,X-LC,25.81,156895.0,16086.0,9445.0,-0.21,11.42,6.02,18.13,19.0,1.70,1.20,23.88,XY25,NTT,REFINERIES
85,UNIONBANK,123.87,163.0,-0.31,M-LC,93.55,173381.0,32541.0,11946.0,-0.04,23.10,6.89,31.59,79.0,2.72,1.33,50.61,XY24,NTT,BANKS
56,PFIZER,4245.50,6318.3,14.35,H-SC,98.92,132296.0,34650.0,13031.0,-0.78,35.48,9.85,48.82,134.0,2.66,1.01,46.97,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,ROUTE,1558.40,2249.26,-39.21,M-SC,35.48,65333.0,-37521.0,83117.0,2.71,-36.48,127.22,44.33,172.0,-0.45,0.50,8.49,SR,ATH,IT
2,ABB,5551.76,7934.00,-32.87,H-LC,74.19,239694.0,23175.0,69727.0,1.90,10.70,29.09,42.91,27.0,0.33,1.83,25.29,AR,NTT,ELECTRICAL
8,ASIANTILES,75.41,137.00,6522.22,M-SC,97.85,72662.0,-21148.0,97767.0,1.79,-22.54,134.55,81.67,214.0,-0.22,0.56,40.71,XR,NTT,CERAMICS
88,VBL,492.64,672.85,-15.75,H-LC,3.23,255954.0,-9086.0,106042.0,1.63,-3.43,41.43,36.58,26.0,-0.09,1.96,9.31,AR,ATH,FMCG
71,SIEMENS,4406.90,7969.85,-7.94,X-LC,67.74,83292.0,-26880.0,115951.0,1.56,-24.40,139.21,80.85,11.0,-0.23,0.64,22.68,AR,ATH,ELECTRICAL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,SIS,477.00,528.00,2208.96,M-SC,75.81,62509.0,-18581.0,27254.0,-2.35,-22.91,43.60,10.69,201.0,-0.68,0.48,26.68,X40N,NTT,MISC
73,SONACOMS,578.05,1006.00,-29.72,M-MC,34.41,90834.0,-10325.0,85220.0,-1.71,-10.21,93.82,74.03,155.0,-0.12,0.69,27.66,AR,BTT,AUTO
14,BERGEPAINT,558.54,680.00,-15.21,H-MC,45.16,182132.0,5075.0,33421.0,-1.28,2.87,18.35,21.75,104.0,0.15,1.39,30.28,XY24,NTT,PAINTS
77,TANLA,917.30,1963.11,-33.79,H-SC,95.70,162394.0,-77021.0,349975.0,-1.02,-32.17,215.51,114.01,113.0,-0.22,1.24,48.94,AR,ATH,IT
6,APOLLOHOSP,6904.43,8285.00,-17.22,H-LC,10.75,102188.0,-1378.0,22083.0,-0.92,-1.33,21.61,20.00,35.0,-0.06,0.78,14.60,AR,BTT,HEALTHCARE


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,1461.46,2275.00,-13.50,X-LC,36.56,277468.0,15867.0,129744.0,-0.27,6.07,46.76,55.67,1.0,0.12,2.12,12.54,X40,BTT,IT
80,TCS,3794.03,4998.00,-19.37,X-LC,15.05,262486.0,-33448.0,127358.0,-0.17,-11.30,48.52,31.73,2.0,-0.26,2.01,5.06,X40,BTT,IT
51,LTIM,5564.16,7197.33,-3.37,X-LC,73.12,186120.0,-14190.0,72978.0,-0.22,-7.08,39.21,29.35,3.0,-0.19,1.42,28.58,X200,ATH,IT
7,ASIANPAINT,2961.56,4250.00,-23.13,X-LC,4.30,152735.0,-48651.0,136270.0,0.12,-24.16,89.22,43.51,5.0,-0.36,1.17,5.35,X40,BTT,PAINTS
35,HINDUNILVR,2413.81,2723.00,-15.01,X-LC,17.20,267008.0,-3339.0,37969.0,0.32,-1.23,14.22,12.81,6.0,-0.09,2.04,10.51,XY25,NTT,FMCG


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,PFIZER,4245.50,6318.30,14.35,H-SC,98.92,132296.0,34650.0,13031.0,-0.78,35.48,9.85,48.82,134.0,2.66,1.01,46.97,X40,ATH,PHARMA
57,POLYCAB,5005.95,7541.35,7.69,H-MC,26.88,102272.0,17171.0,25926.0,0.59,20.18,25.35,50.65,90.0,0.66,0.78,28.56,X40N,ATH,CABLES
85,UNIONBANK,123.87,163.00,-0.31,M-LC,93.55,173381.0,32541.0,11946.0,-0.04,23.10,6.89,31.59,79.0,2.72,1.33,50.61,XY24,NTT,BANKS
4,ADANIPORTS,1103.69,1583.00,3.60,M-LC,63.98,103802.0,25440.0,8595.0,0.36,32.46,8.28,43.43,70.0,2.96,0.79,38.83,XY24,NTT,MISC
50,LICI,785.05,983.00,-2.96,H-LC,89.25,237609.0,42917.0,6178.0,0.16,22.04,2.60,25.21,31.0,6.95,1.82,31.39,XY25,NTT,INSURANCE


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SAIL,130.64,228.00,101.81,M-MC,87.10,229267.0,4305.0,163353.0,0.02,1.91,71.25,74.53,170.0,0.03,1.75,33.11,XY24,BTT,STEEL
52,LUXIND,1448.19,2442.00,38.95,M-SC,69.89,144010.0,6432.0,87976.0,-0.74,4.68,61.09,68.62,183.0,0.07,1.10,24.85,X40N,NTT,TEXTILES
56,PFIZER,4245.50,6318.30,14.35,H-SC,98.92,132296.0,34650.0,13031.0,-0.78,35.48,9.85,48.82,134.0,2.66,1.01,46.97,X40,ATH,PHARMA
57,POLYCAB,5005.95,7541.35,7.69,H-MC,26.88,102272.0,17171.0,25926.0,0.59,20.18,25.35,50.65,90.0,0.66,0.78,28.56,X40N,ATH,CABLES
18,CAMS,3643.00,5250.99,6.22,H-SC,86.02,119462.0,17458.0,27560.0,0.43,17.12,23.07,44.14,112.0,0.63,0.91,36.14,XR,ATH,MISC


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,3643.00,5250.99,6.22,H-SC,86.02,119462.0,17458.0,27560.0,0.43,17.12,23.07,44.14,112.0,0.63,0.91,36.14,XR,ATH,MISC
12,BANKINDIA,113.49,190.00,-27.01,M-MC,70.97,142068.0,12916.0,74145.0,0.47,10.00,52.19,67.42,156.0,0.17,1.09,38.17,XR,NTT,BANKS
39,INDIAMART,2327.09,4871.06,-51.28,H-SC,60.22,127020.0,3684.0,131148.0,-0.88,2.99,103.25,109.32,108.0,0.03,0.97,25.49,AR,ATH,MISC
92,WIPRO,243.46,420.00,-10.16,M-LC,33.33,153661.0,2716.0,106733.0,-0.04,1.80,69.46,72.51,68.0,0.03,1.17,9.66,XR,NTT,IT
1,AARTIIND,487.04,919.00,39.14,M-SC,62.37,79960.0,-1376.0,73515.0,-0.59,-1.69,91.94,88.69,205.0,-0.02,0.61,34.84,XR,NTT,CHEMICALS


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MASFIN,326.60,399.5,-25.02,H-SC,57.53,85260.0,-12720.0,34590.0,-0.18,-12.98,40.57,22.32,127.0,-0.37,0.65,23.66,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.0,3.60,M-LC,63.98,103802.0,25440.0,8595.0,0.36,32.46,8.28,43.43,70.0,2.96,0.79,38.83,XY24,NTT,MISC
87,VALIANTORG,512.64,838.0,-371.37,H-SC,100.00,115427.0,-17859.0,102453.0,-0.30,-13.40,88.76,63.47,130.0,-0.17,0.88,93.48,XR,NTT,CHEMICALS
84,UJJIVANSFB,52.77,60.0,38.86,M-SC,59.14,120825.0,-21654.0,41177.0,0.56,-15.20,34.08,13.70,221.0,-0.53,0.92,42.61,X40N,NTT,BANKS
56,PFIZER,4245.50,6318.3,14.35,H-SC,98.92,132296.0,34650.0,13031.0,-0.78,35.48,9.85,48.82,134.0,2.66,1.01,46.97,X40,ATH,PHARMA


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VALIANTORG,512.64,838.00,-371.37,H-SC,100.00,115427.0,-17859.0,102453.0,-0.30,-13.40,88.76,63.47,130.0,-0.17,0.88,93.48,XR,NTT,CHEMICALS
56,PFIZER,4245.50,6318.30,14.35,H-SC,98.92,132296.0,34650.0,13031.0,-0.78,35.48,9.85,48.82,134.0,2.66,1.01,46.97,X40,ATH,PHARMA
8,ASIANTILES,75.41,137.00,6522.22,M-SC,97.85,72662.0,-21148.0,97767.0,1.79,-22.54,134.55,81.67,214.0,-0.22,0.56,40.71,XR,NTT,CERAMICS
81,TITAGARH,1097.23,1548.00,-7.32,H-SC,96.77,162934.0,-29081.0,107960.0,-0.31,-15.15,66.26,41.08,128.0,-0.27,1.25,37.90,XY24,NTT,ENGINEERING
77,TANLA,917.30,1963.11,-33.79,H-SC,95.70,162394.0,-77021.0,349975.0,-1.02,-32.17,215.51,114.01,113.0,-0.22,1.24,48.94,AR,ATH,IT


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.79
1,25,42.15
2,50,70.56


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    31.58
AR      15.08
XY25    14.67
X40N    13.27
X40     12.45
XR      10.37
X200     1.42
SR       1.14
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    19.99
X-LC    18.07
M-SC    17.40
H-LC     9.25
H-MC     8.50
M-LC     7.70
X-MC     5.53
M-MC     5.52
X-SC     2.76
L-SC     2.30
L-LC     1.13
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.52,-1.39,33.14
IT,10.18,-17.12,80.73
BANKS,9.66,-10.97,61.98
MISC,6.52,-26.78,82.61
FINANCE,5.81,-25.59,85.22
PAINTS,5.28,-19.04,54.27
INSURANCE,5.27,3.42,25.44
ELECTRICAL,5.15,-4.27,49.50
HEALTHCARE,4.34,-4.66,35.89


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2844286.0,26
AR,1249861.0,14
X40N,1210694.0,17
XR,1095940.0,14
X40,778458.0,9
XY25,460924.0,10
SR,196036.0,2
X200,72978.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2223015.0,20
M-SC,2040154.0,21
X-LC,848537.0,11
M-MC,545224.0,5
H-MC,512825.0,8
X-SC,347590.0,4
X-MC,309537.0,5
M-LC,298417.0,6
L-SC,298338.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      929905.0      6
M-SC       XY24      759242.0      6
H-SC       AR        540666.0      3
           X40N      499536.0      6
X-LC       X40       482900.0      4
M-SC       X40N      472138.0      6
M-MC       XY24      385859.0      3
M-SC       XR        289593.0      4
H-SC       XR        239877.0      4
H-LC       AR        236396.0      5
M-SC       X40       220015.0      2
M-LC       XY24      176862.0      4
H-MC       XY24      172796.0      2
X-LC       AR        166167.0      2
L-SC       XR        158116.0      2
X-MC       XY24      139837.0      2
X-LC       XY25      126492.0      4
M-SC       XY25      121800.0      1
X-SC       X40N      119722.0      2
           XR        114949.0      1
           SR        112919.0      1
H-MC       XR        112527.0      1
M-LC       XR        106733.0      1
M-SC       AR         94249.0      1
X-MC       X40N       93372.0      2
M-MC       AR         85220.0      1
M-SC       SR         83117.0      1
L-SC       XY24       78604.0      1
X-MC       XY25       76328.0      1
M-MC       XR         74145.0      1
H-MC       XY25       73519.0      1
X-LC       X200       72978.0      1
H-MC       AR         65545.0      1
           X40        62512.0      2
L-SC       AR         61618.0      1
L-LC       XY25       41785.0      1
H-MC       X40N       25926.0      1
M-LC       XY25       14822.0      1
H-SC       X40        13031.0      1
H-LC       XY25        6178.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 55.0 seconds
